# Clustering Crypto

In [17]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [89]:
# Load the crypto_data.csv dataset.
file_path="Resources/crypto_data.csv"
crypto_df=pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [90]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df.shape

(1144, 6)

In [82]:
# Keep all the cryptocurrencies that have a working algorithm.
crypo_df = crypo_df[crypo_df['Algorithm'] != 'n/a']
crypo_df.shape

(1144, 6)

In [91]:
# Remove the "IsTrading" column. 
crypto_df=crypto_df.drop(['IsTrading'],axis=1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [92]:
# Remove rows that have at least 1 null value.
crypto_df=crypto_df.dropna()
crypto_df.shape

(685, 5)

In [93]:
# Keep the rows where coins are mined.
crypto_df=crypto_df[crypto_df["TotalCoinsMined"]>0]
crypto_df.shape

(532, 5)

In [68]:
# Create a new DataFrame that holds only the cryptocurrencies names.
name_df=pd.DataFrame(df['CoinName'])
name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [94]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [95]:
# Use get_dummies() to create variables for text features.
X= pd.get_dummies(crypto_df,columns=["Algorithm","ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.14159E+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710816 -0.15287029 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.14500899 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494542  4.48942055 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217936 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694816 -0.15255997 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710535 -0.15285551 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [97]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.31829589,  0.94237243, -0.5566358 ],
       [-0.30166635,  0.94294481, -0.55703102],
       [ 2.29902899,  1.7574017 , -0.63356046],
       ...,
       [ 0.32167799, -2.25785865,  0.36656557],
       [-0.17754619, -1.81842381,  0.38497379],
       [-0.28332472,  0.85012623, -0.23122337]])

In [105]:
# Create a DataFrame with the three principal components.
x_pca_df= pd.DataFrame(data=X_pca, columns=["principal component 1","principal component 2","principal component 3"],index=crypto_df.index
)
x_pca_df.head()

,principal component 1,principal component 2,principal component 3
42,-0.318296,0.942372,-0.556636
404,-0.301666,0.942945,-0.557031
1337,2.299029,1.757402,-0.633560
BTC,-0.153362,-1.284528,0.219475
ETH,-0.163273,-2.026180,0.413511


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [106]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(x_pca_df)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)




:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [107]:
# Initialize the K-Means model.
model= KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(x_pca_df)

# Predict clusters
predictions = model.predict(x_pca_df)
print (predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 3 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 3 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 3 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 0 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [108]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, x_pca_df], sort=False, axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = name_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)

clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.318296,0.942372,-0.556636,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.301666,0.942945,-0.557031,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,2.299029,1.757402,-0.633560,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.153362,-1.284528,0.219475,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.163273,-2.026180,0.413511,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.154338,-1.173590,-0.006423,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.410301,1.229032,-0.520161,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.149852,-2.271572,0.368267,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.161718,-2.026235,0.413485,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.177545,-1.818424,0.384974,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [113]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name='CoinName',
    hover_data=['Algorithm']
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [116]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined', 'Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [117]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [118]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
tradable = MinMaxScaler()
data = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
tradable_scaled = tradable.fit_transform(data)
tradable_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585514e-03],
       [3.14159000e-01, 2.95755050e-02],
       ...,
       [1.40022261e-03, 9.90134792e-04],
       [2.10000000e-05, 7.37027937e-06],
       [1.00000000e-06, 1.29582244e-07]])

In [120]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=tradable_scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141590e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [121]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols='CoinName')


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)